# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 81 new papers today
          19 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/18 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.10919


extracting tarball to tmp_2302.10919...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.10932


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2302.10919/universe-2041627-english.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2302.11392...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.11397


extracting tarball to tmp_2302.11397...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.11549


/tmp/ipykernel_1990/4030337529.py:34: LatexWarning: 2302.11549 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10943-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10943) | **The growth of brightest cluster galaxies in the TNG300  simulation:dissecting the contributions from mergers and in situ star  formation**  |
|| Daniel Montenegro-Taborda, et al. -- incl., <mark>Annalisa Pillepich</mark> |
|*Appeared on*| *2023-02-23*|
|*Comments*| *19 pages, 11 figures. Accepted for publication in MNRAS*|
|**Abstract**| We investigate the formation of brightest cluster galaxies (BCGs) in the TNG300 cosmological simulation of the IllustrisTNG project. Our cluster sample consists of 700 haloes with $M_{200} \geq 5 \times 10^{13} \, \mathrm{M}_{\odot}$ at $z=0$, along with their progenitors at earlier epochs. This includes 280 systems with $M_{200} \geq 10^{14} \, \mathrm{M}_{\odot}$ at $z=0$, as well as three haloes with $M_{200} \geq 10^{15} \, \mathrm{M}_{\odot}$. We find that the stellar masses and star formation rates of our simulated BCGs are in good agreement with observations at $z \lesssim 0.4$, and that they have experienced, on average, $\sim$2 ($\sim$3) major mergers since $z=1$ ($z=2$). Separating the BCG from the intracluster light (ICL) by means of a fixed 30 kpc aperture, we find that the fraction of stellar mass contributed by ex situ (i.e. accreted) stars at $z=0$ is approximately 70, 80, and 90 per cent for the BCG, BCG+ICL, and ICL, respectively. Tracking our simulated BCGs back in time using the merger trees, we find that they became dominated by ex situ stars at $z \sim $1-2, and that half of the stars that are part of the BCG at $z=0$ formed early ($z \sim 3$) in other galaxies, but `assembled' onto the BCG until later times ($z \approx 0.8$ for the whole sample, $z \approx 0.5$ for BCGs in $M_{200} \geq 5 \times 10^{14} \, \mathrm{M}_{\odot}$ haloes). Finally, we show that the stellar mass profiles of BCGs are often dominated by ex situ stars at all radii, with stars from major mergers being found closer to the centre, while stars that were tidally stripped from other galaxies dominate the outer regions. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10919-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10919) | **Multiverse Predictions for Habitability: Element Abundances**  |
|| McCullen Sandora, et al. -- incl., <mark>Geraint F. Lewis</mark> |
|*Appeared on*| *2023-02-23*|
|*Comments*| *27 pages, 5 figures*|
|**Abstract**| We investigate the dependence of elemental abundances on physical constants, and the implications this has for the distribution of complex life for various proposed habitability criteria. We consider three main sources of abundance variation: differing supernova rates, alpha burning in massive stars, and isotopic stability, and how each affects the metal-to-rock ratio and the abundances of carbon, oxygen, nitrogen, phosphorus, sulfur, silicon, magnesium, and iron. Our analysis leads to several predictions for which habitability criteria are correct by determining which ones make our observations of the physical constants, as well as a few other observed features of our universe, most likely. Our results indicate that carbon-rich or carbon-poor planets are uninhabitable, slightly magnesium-rich planets are habitable, and life does not depend on nitrogen abundance too sensitively. We also find suggestive but inconclusive evidence that metal-rich planets and phosphorus-poor planets are habitable. These predictions can then be checked by probing regions of our universe that closely resemble normal environments in other universes. If any of these predictions are found to be wrong, the multiverse scenario would predict that the majority of observers are born in universes differing substantially from ours, and so can be ruled out, to varying degrees of statistical significance. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10932-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10932) | **Scary Barbie: An Extremely Energetic, Long-Duration Tidal Disruption  Event Candidate Without a Detected Host Galaxy at z = 0.995**  |
|| Bhagya M. Subrayan, et al. -- incl., <mark>Kyoung-Soo Lee</mark>, <mark>David Matthews</mark> |
|*Appeared on*| *2023-02-23*|
|*Comments*| *13 pages, 4 figures, 1 Table; Submitted to The Astrophysical Journal Letters*|
|**Abstract**| We report multi-wavelength observations and characterization of the ultraluminous transient AT 2021lwx (ZTF20abrbeie; aka ``Barbie'') identified in the alert stream of the Zwicky Transient Facility (ZTF) using a Recommender Engine For Intelligent Transient Tracking (REFITT) filter on the ANTARES alert broker. From a spectroscopically measured redshift of 0.995, we estimate a peak observed pseudo-bolometric luminosity of log L$_{\text{max}} = 45.7$ erg s$^{-1}$ from slowly fading ztf-$\it{g}$ and ztf-$r$ light curves spanning over 1000 observer-frame days. The host galaxy is not detected in archival Pan-STARRS observations ($g > 23.3$ mag), implying a lower limit to the outburst amplitude of more than 5 mag relative to the quiescent host galaxy. Optical spectra from Lick and Keck Observatories exhibit strong emission lines with narrow cores from the H Balmer series and ultraviolet semi-forbidden lines of Si III] $\lambda$1892, C III] $\lambda$1909, and C II] $\lambda$2325. Typical nebular lines in AGN spectra from ions such as [O II] and [O III] are not detected. These spectral features, along with the smooth light curve that is unlike most AGN flaring activity, and the luminosity that exceeds any observed or theorized supernova, lead us to conclude that AT 2021lwx is most likely an extreme tidal disruption event (TDE). Modeling of ZTF photometry with MOSFiT suggests that the TDE was between a $\approx 14 M_{\odot}$ star and a supermassive black hole of mass $M_{\text{BH}} \sim$ $10^{8} M_{\odot}$. Continued monitoring of the still-evolving light curve along with deep imaging of the field once AT 2021lwx has faded can test this hypothesis and potentially detect the host galaxy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11324-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11324) | **The AGNIFS survey: spatially resolved observations of hot molecular and  ionised outflows in nearby active galaxies**  |
|| R. A. Riffel, et al. -- incl., <mark>R. I. Davies</mark>, <mark>T. C. Fischer</mark> |
|*Appeared on*| *2023-02-23*|
|*Comments*| *37 pages, accepted for publication in MNRAS*|
|**Abstract**| We present the hot molecular and warm ionised gas kinematics for 33 nearby ($0.001\lesssim z\lesssim0.056$) X-ray selected active galaxies using the H$_2 2.1218 \mu$m and Br$\gamma$ emission lines observed in the K-band with the Gemini Near-Infrared Field Spectrograph (NIFS). The observations cover the inner 0.04$-$2 kpc of each AGN at spatial resolutions of 4$-$250 pc with a velocity resolution of $\sigma_{\rm inst}\approx$20 ${\rm km s^{-1}}$. We find that 31 objects (94 per cent) present a kinematically disturbed region (KDR) seen in ionised gas, while such regions are observed in hot molecular gas for 25 galaxies (76 per cent). We interpret the KDR as being due to outflows with masses of 10$^2-$10$^7$ M$_\odot$ and 10$^0-$10$^4$ M$_\odot$ for the ionised and hot molecular gas, respectively. The ranges of mass-outflow rates ($\dot{M}_{\rm out}$) and kinetic power ($\dot{E}_{\rm K}$) of the outflows are 10$^{-3}-$10$^{1}$ M$_\odot$yr$^{-1}$ and $\sim$10$^{37}$$-$10$^{43}$ erg s$^{-1}$ for the ionised gas outflows, and 10$^{-5}$$-$10$^{-2}$ M$_\odot$ yr$^{-1}$ and 10$^{35}$$-$10$^{39}$ erg s$^{-1}$ for the hot molecular gas outflows. The median coupling efficiency in our sample is $\dot{E}_{K}/L_{\rm bol}\approx1.8\times10^{-3}$ and the estimated momentum fluxes of the outflows suggest they are produced by radiation-pressure in low-density environment, with possible contribution from shocks. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11392-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11392) | **Photometric follow-up of 43 new eclipsing white dwarf plus main-sequence  binaries from the ZTF survey**  |
|| <mark>Alex J. Brown</mark>, et al. -- incl., <mark>Matthew J. Green</mark> |
|*Appeared on*| *2023-02-23*|
|*Comments*| *12 pages with a 5 page appendix and 14 figures. Accepted for publication in MNRAS*|
|**Abstract**| Wide-field time-domain photometric sky surveys are now finding hundreds of eclipsing white dwarf plus M dwarf binaries, a population encompassing a wealth of information and potential insight into white dwarf and close binary astrophysics. Precise follow-up observations are essential in order to fully constrain these systems and capitalise on the power of this sample. We present the first results from our program of high-speed, multi-band photometric follow-up. We develop a method to measure temperatures, (model-dependent) masses, and radii for both components from the eclipse photometry alone and characterize 34 white dwarf binaries, finding general agreement with independent estimates using an alternative approach while achieving around a factor of two increase in parameter precision. In addition to these parameter estimates, we discover a number of interesting systems -- finding four with sub-stellar secondaries, doubling the number of eclipsing examples, and at least six where we find the white dwarf to be strongly magnetic, making these the first eclipsing examples of such systems and key to investigating the mechanism of magnetic field generation in white dwarfs. We also discover the first two pulsating white dwarfs in detached and eclipsing post-common-envelope binaries -- one with a low-mass, likely helium core, and one with a relatively high mass, towards the upper end of the known sample of ZZ Cetis. Our results demonstrate the power of eclipse photometry, not only as a method of characterising the population, but as a way of discovering important systems that would have otherwise been missed by spectroscopic follow-up. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11397-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11397) | **Measurement of telescope transmission using a Collimated Beam Projector**  |
|| Nicholas Mondrik, et al. -- incl., <mark>Joseph P. Rice</mark> |
|*Appeared on*| *2023-02-23*|
|*Comments*| **|
|**Abstract**| With the increasingly large number of type Ia supernova being detected by current-generation survey telescopes, and even more expected with the upcoming Rubin Observatory Legacy Survey of Space and Time, the precision of cosmological measurements will become limited by systematic uncertainties in flux calibration rather than statistical noise. One major source of systematic error in determining SNe Ia color evolution (needed for distance estimation) is uncertainty in telescope transmission, both within and between surveys. We introduce here the Collimated Beam Projector (CBP), which is meant to measure a telescope transmission with collimated light. The collimated beam more closely mimics a stellar wavefront as compared to flat-field based instruments, allowing for more precise handling of systematic errors such as those from ghosting and filter angle-of-incidence dependence. As a proof of concept, we present CBP measurements of the StarDICE prototype telescope, achieving a standard (1 sigma) uncertainty of 3 % on average over the full wavelength range measured with a single beam illumination. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11020-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11020) | **Comparisons of the core and mantle compositions of earth analogs from  different terrestrial planet formation scenarios**  |
|| Jesse T. Gu, et al. -- incl., <mark>Rebecca A. Fischer</mark> |
|*Appeared on*| *2023-02-23*|
|*Comments*| **|
|**Abstract**| The chemical compositions of Earth's core and mantle provide insight into the processes that led to their formation. N-body simulations, on the other hand, generally do not contain chemical information, and seek to only reproduce the masses and orbits of the terrestrial planets. These simulations can be grouped into four potentially viable scenarios of Solar System formation (Classical, Annulus, Grand Tack, and Early Instability) for which we compile a total of 433 N-body simulations. We relate the outputs of these simulations to the chemistry of Earth's core and mantle using a melt-scaling law combined with a multi-stage model of core formation. We find the compositions of Earth analogs to be largely governed by the fraction of equilibrating embryo cores and the initial embryo masses in N-body simulations. Simulation type may be important when considering magma ocean lifetimes, where Grand Tack simulations have the largest amounts of material accreted after the last giant impact. However, we cannot rule out any accretion scenarios or initial embryo masses due to the sensitivity of Earth's mantle composition to different parameters and the stochastic nature of N-body simulations. Comparing the last embryo impacts experienced by Earth analogs to specific Moon-forming scenarios, we find the characteristics of the Moon-forming impact are dependent on the initial conditions in N-body simulations where larger initial embryo masses promote larger and slower Moon-forming impactors. Mars-sized initial embryos are most consistent with the canonical hit-and-run scenario onto a solid mantle. Our results suggest that constraining the fraction of equilibrating impactor core and the initial embryo masses in N-body simulations could be significant for understanding both Earth's accretion history and characteristics of the Moon-forming impact. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11549-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11549) | **Quasi-periodic pulsations in solar flares: a key diagnostic of energy  release on the Sun**  |
|| Andrew Inglis, et al. -- incl., <mark>Bo Li</mark> |
|*Appeared on*| *2023-02-23*|
|*Comments*| *White paper submitted to the Decadal Survey for Solar and Space Physics (Heliophysics) 2024-2033. 8 pages, 3 figures, 1 table*|
|**Abstract**| Solar flares are among the most powerful and disruptive events in our solar system, however the physical mechanisms driving and transporting this energetic release are not fully understood. An important signature associated with flare energy release is highly variable emission on timescales of sub-seconds to minutes which often exhibit oscillatory behaviour, features collectively known as quasi-periodic pulsations (QPPs). To fully identify the driving mechanism of QPPs, exploit their potential as a diagnostic tool, and incorporate them into our understanding of solar and stellar flares, new observational capabilities and initiatives are required. There is a clear community need for flare-focused, rapid cadence, high resolution, multi-wavelength imaging of the Sun, with high enough sensitivity and dynamic range to observe small fluctuations in intensity in the presence of a large overall intensity. Furthermore, multidisciplinary funding and initiatives are required to narrow the gap between numerical models and observations. QPPs are direct signatures of the physics occurring in flare magnetic reconnection and energy release sites and hence are critical to include in a unified flare model. Despite significant modelling and theoretical work, no single mechanism or model can fully explain the presence of QPPs in flares. Moreover, it is also likely that QPPs fall into different categories that are produced by different mechanisms. At present we have insufficient information to observationally distinguish between mechanisms. The motivation to understand QPPs is strengthened by the geo-effectiveness of flares on the Earth's ionosphere, and by the fact that stellar flares exhibit similar QPP signatures. QPPs present a golden opportunity to better understand flare physics and exploit the solar-stellary analogy, benefiting both astrophysics, heliophysics, and the solar-terrestrial connection. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.10943.md
    + _build/html/tmp_2302.10943/./figs_BCGs/formation_history_Mstar_with-inrad_median2_1e14.png
    + _build/html/tmp_2302.10943/./figs_BCGs/history_SFR-sSFR_vs_z_median_observation_v3.png
    + _build/html/tmp_2302.10943/./figs_BCGs/history_SFR-sSFR_median-v4-AllAperturesFixed_simulation_lower_lim.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\daniel}[1]{\textbf{{\color{Emerald}{Daniel: #1}}}}$
$\newcommand{\vicente}[1]{\textbf{{\color{violet}{Vicente: #1}}}}$
$\newcommand{\vlad}[1]{\textbf{{\color{cyan}{VAR: #1}}}}$
$\newcommand{\aldo}[1]{\textbf{{\color{orange}{Aldo: #1}}}}$
$\newcommand{\ap}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\todo}[1]{\textcolor{red}{TODO: please #1}}$
$\newcommand{\Msun}{{\rm M}_{\odot}}$
$\newcommand{\Mtwo}{M_{\rm 200}}$
$\newcommand{\Mfive}{M_{\rm 500}}$
$\newcommand{\Rtwo}{R_{\rm 200}}$
$\newcommand{\Rfive}{R_{\rm 500}}$
$\newcommand{\facc}{f_{\rm acc}}$
$\newcommand{\fex}{f_{\rm ex}}$
$\newcommand{\krot}{\kappa_{\rm rot}}$
$\newcommand{\thefootnote}{\fnsymbol{footnote}}$
$\newcommand{\thefootnote}{\arabic{footnote}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\daniel$}[1]{\textbf{{\color{Emerald}{Daniel: #1}}}}$
$\newcommand{$\vicente$}[1]{\textbf{{\color{violet}{Vicente: #1}}}}$
$\newcommand{$\vlad$}[1]{\textbf{{\color{cyan}{VAR: #1}}}}$
$\newcommand{$\aldo$}[1]{\textbf{{\color{orange}{Aldo: #1}}}}$
$\newcommand{$\ap$}[1]{\textcolor{magenta}{#1}}$
$\newcommand{$\todo$}[1]{\textcolor{red}{TODO: please #1}}$
$\newcommand{$\Msun$}{{\rm M}_{\odot}}$
$\newcommand{$\Mtwo$}{M_{\rm 200}}$
$\newcommand{$\Mfive$}{M_{\rm 500}}$
$\newcommand{$\Rtwo$}{R_{\rm 200}}$
$\newcommand{$\Rfive$}{R_{\rm 500}}$
$\newcommand{$\facc$}{f_{\rm acc}}$
$\newcommand{$\fex$}{f_{\rm ex}}$
$\newcommand{$\krot$}{\kappa_{\rm rot}}$
$\newcommand{$\thefootnote$}{\fnsymbol{footnote}}$
$\newcommand{$\thefootnote$}{\arabic{footnote}}$</div>



<div id="title">

#  star formation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.10943-b31b1b.svg)](https://arxiv.org/abs/2302.10943)<mark>Appeared on: 2023-02-23</mark> - _19 pages, 11 figures. Accepted for publication in MNRAS_

</div>
<div id="authors">

Daniel Montenegro-Taborda, et al. -- incl., <mark><mark>Annalisa Pillepich</mark></mark>

</div>
<div id="abstract">

**Abstract:** We investigate the formation of brightest cluster galaxies (BCGs) in the TNG300 cosmological simulation of the IllustrisTNG project. Our cluster sample consists of 700 haloes with$\Mtwo \geq 5 \times 10^{13}   $\Msun$$at$z=0$, along with their progenitors at earlier epochs. This includes 280 systems with$\Mtwo \geq 10^{14}   $\Msun$$at$z=0$, as well as three haloes with$\Mtwo \geq 10^{15}   $\Msun$$. We find that the stellar masses and star formation rates of our simulated BCGs are in good agreement with observations at$z \lesssim 0.4$, and that they have experienced, on average,$\sim$2 ($\sim$3) major mergers since$z=1$($z=2$). Separating the BCG from the intracluster light (ICL) by means of a fixed 30 kpc aperture, we find that the fraction of stellar mass contributed by\textit{ex situ}(i.e. accreted) stars at$z=0$is approximately 70, 80, and 90 per cent for the BCG, BCG+ICL, and ICL, respectively. Tracking our simulated BCGs back in time using the merger trees, we find that they became dominated by\textit{ex situ}stars at$z \sim $1--2, and that half of the stars that are part of the BCG at$z=0$formed early ($z \sim 3$) in other galaxies, but `assembled' onto the BCG until later times ($z $\ap$prox 0.8$for the whole sample,$z $\ap$prox 0.5$for BCGs in$\Mtwo \geq 5 \times 10^{14}   $\Msun$$haloes). Finally, we show that the stellar mass profiles of BCGs are often dominated by\textit{ex situ}stars at all radii, with stars from major mergers being found closer to the centre, while stars that were tidally stripped from other galaxies dominate the outer regions.

</div>

<div id="div_fig1">

<img src="tmp_2302.10943/./figs_BCGs/formation_history_Mstar_with-inrad_median2_1e14.png" alt="Fig4" width="100%"/>

**Figure 4. -** Median mass histories of BCGs in the TNG300 simulation selected at $z=0$ with $\Mtwo \geq 10^{14}   $\Msun$$ and tracked back in time along the main branch of their merger trees. The solid red line shows the evolution of the total stellar mass that is gravitationally bound to the central main galaxy, while the solid purple line shows the stellar mass of the central main galaxy within $2 r_{\rm half, \ast}$. The solid orange line shows the mass evolution their host haloes' total mass, quantified by $\Mtwo$, while the solid green line shows the mass evolution of the supermassive BH hosted at the centre of the BCG. The shaded regions indicate the corresponding 16th to 84th percentile ranges, i.e. the galaxy-to-galaxy variation. The dashed lines represent the median mass histories for $\Mtwo$, $M_{\ast, 2 r_{\rm half, \ast}}$, and $M_{\rm BH}$ according to the semi-empirical model of Rodríguez-Puebla et al. (in preparation). Note that, unlike the halo mass history, the shape of the stellar mass history displays an inflection at $z \sim 3$--$4$, such that the stellar mass growth rate slows down after this transition epoch. (*fig:mass-vs-z*)

</div>
<div id="div_fig2">

<img src="tmp_2302.10943/./figs_BCGs/history_SFR-sSFR_vs_z_median_observation_v3.png" alt="Fig2" width="100%"/>

**Figure 2. -** SFRs of BCGs across cosmic time according to the TNG300 simulation. In the top (bottom) panels, we show SFRs (specific SFRs). Median SFR (top) and specific SFR (bottom) as function of redshift, shown for TNG300 BCGs \textit{selected at each redshift} with $\Mtwo \geq 10^{14}   $\Msun$$. The blue curves show measurements within 50 kpc from TNG300 including galaxies with three different SFR lower limits: 0.1 (solid), 1 (dashed), and 10 (dotted) $\Msun{\rm yr^{-1}}$. The black dotted rectangles, grey crosses and green diamonds represent observational estimates by \protect\cite{mcdonald2016SFR}, \protect\cite{Bonaventura2017} and \protect\cite{Orellana-Gonzalez+2022}, respectively, for SZ- or optically-selected clusters, and thus including a diversity of galaxy and halo mass distributions at different redshifts.
	\label{fig:SFR-vs-z_observation} (*fig:SFR-vs-z_observation*)

</div>
<div id="div_fig3">

<img src="tmp_2302.10943/./figs_BCGs/history_SFR-sSFR_median-v4-AllAperturesFixed_simulation_lower_lim.png" alt="Fig3" width="100%"/>

**Figure 3. -** SF histories of BCGs across cosmic time according to the TNG300 simulation. In the top (bottom) panels, we show SFRs (specific SFRs).
	Median SFR (top) and specific SFR (bottom) as function of redshift, shown for the main progenitors of simulated BCGs selected at $z=0$ with $\Mtwo \geq 10^{14}   $\Msun$$. The solid curves show measurements from TNG300 within 50 kpc (blue) and for the entire central galaxy (red), as well as for the entire central galaxy in Illustris original (green). The blue shaded regions indicate the 16th to 84th percentile range of the $r < 50$ kpc TNG300 measurements. The dashed grey line shows results from the semi-empirical model of Rodríguez-Puebla et al. (in preparation). Null SFR values have been replaced with the minimum measurable values as discussed in \protect\cite{donnari2019}.
	\label{fig:SFR-vs-z_simulation} (*fig:SFR-vs-z_simulation*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

213  publications in the last 7 days.
	 _build/html/2302.10943.md
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> daily </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


# Debugging papers